In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data
from yahooquery import Ticker

In [ ]:
#sivbq = Ticker("SIVBQ")
stock_tickers = Ticker(["SIVBQ","JPM","FRCB",
                        "WFC","BAC","DFS",
                        "MS","PNC","CS",
                        "DB","COF","C",
                        "KEY","BNY","GS",
                        "HSBC","MTB","TD",
                        "FHN","OVLY","PACW",
                        "WAL","SBNY","ZION",
                        "CMA","FCNCA","EWBC",
                        "CFG","MCBC","NYCB"])

#for ticker in stock_tickers:
 #   stock = yf.Ticker(ticker)

#stock_tickers.key_stats
history_df = stock_tickers.history()
history_df["market_cap"] = history_df["volume"]*history_df["adjclose"]


#hist = sivbq.history(period="1mo")
#shares = sivbq.get_shares_full(start="2017-01-01", end=None)
#type(shares)

# Quarterly highs and lows for historical data between Jan 1 2017 - March 1 2022
#stock_df_svb = sivbq.history(period ="max", interval="3mo", start="2017-01-01", end="2022-04-01")
#stock_df_svb



In [36]:
#cleanup
import os
import pandas as pd
path_to_parent = "/CSV"
current_path = os.getcwd()


balance_sheet_df = pd.DataFrame()
cashflow_statement_df = pd.DataFrame()
income_statement_df = pd.DataFrame()

copy_df = pd.DataFrame()

for folder in os.listdir(current_path+path_to_parent):
  
    for csv_file in os.listdir(os.path.join(current_path+path_to_parent, folder)):
        print(os.getcwd())
        if csv_file == "balance_sheet_quarterly.csv":
            copy_df = pd.read_csv(current_path+path_to_parent+'/'+folder+'/'+csv_file)

        

        
# iterate through all *.csv files per folder in CSV folder (quarterly info only)



""
